# Level 2 - Week 4 - 01 Context Assembly and Grounding

**Estimated time:** 60-90 minutes

## Learning Objectives

- Assemble context blocks from hits
- Control context length
- Keep doc_id and chunk_id visible


## Overview

Context assembly controls what the model can use.
Keep it structured and deterministic.

## Practice Steps

- Assemble a CONTEXT block from hits.
- Enforce a max length.


### Sample code

Structured context assembly with metadata.


In [ ]:
def assemble_context(hits: list[dict], max_chars: int = 800) -> str:
    parts = []
    total = 0
    for idx, h in enumerate(hits, start=1):
        text = h.get('text', '')
        entry = f"[{idx}] doc_id={h.get('doc_id')} chunk_id={h.get('chunk_id')} text="{text}""
        if total + len(entry) > max_chars:
            break
        parts.append(entry)
        total += len(entry)
    return '
'.join(parts)


### Student fill-in

Provide hits and inspect the context output.


In [ ]:
hits = [
    {'doc_id': 'doc-1', 'chunk_id': 'c1', 'text': 'Example evidence 1'},
    {'doc_id': 'doc-2', 'chunk_id': 'c2', 'text': 'Example evidence 2'},
]

# TODO: adjust hits and max_chars
print(assemble_context(hits, max_chars=200))


## Self-check

- Are chunk_id values preserved in the context?
- Is length capped?


Legacy practice content from practice.ipynb

# Level 2 — Week 4 Practice: RAG v1 (Citations + Refusal)

**Estimated time:** 60–90 minutes

## Learning Objectives

- Format citations with doc_id + chunk_id
- Assemble answer + citations payloads
- Implement refusal/clarification rules
- Keep responses structured and testable


Legacy practice content from practice.ipynb

## Overview

This week assembles a minimal RAG response contract. You will format citations
and handle refusal/clarification when evidence is missing.

You will:

1. Format citations from retrieval hits.
2. Assemble a response payload (answer + citations).
3. Add refusal/clarification rules.

## Practice Steps

- Use the provided stub hits.
- Build the response contract.
- Add a simple refusal condition.


Legacy practice content from practice.ipynb

### Task 4.1: Format citations

Use the helper to format citations from retrieval hits.
Then build a response payload with answer + citations.


In [ ]:
# Legacy practice content
def format_citations(hits):
    out = []
    for h in hits:
        out.append(
            {
                "doc_id": h.get("doc_id"),
                "chunk_id": h.get("chunk_id"),
                "snippet": (h.get("text") or "")[:200],
            }
        )
    return out

stub_hits = [
    {"doc_id": "doc-1", "chunk_id": "c1", "text": "Evidence text 1"},
    {"doc_id": "doc-2", "chunk_id": "c4", "text": "Evidence text 2"},
]

citations = format_citations(stub_hits)
response = {"answer": "Draft answer based on evidence.", "citations": citations}
print(response)


Legacy practice content from practice.ipynb

### Task 4.2: Refusal / clarification rule

Implement a simple rule: if there are no citations, return a refusal payload.


In [ ]:
# Legacy practice content
def build_response(answer: str, citations: list[dict]) -> dict:
    if not citations:
        return {"ok": False, "error": "insufficient evidence"}
    return {"ok": True, "answer": answer, "citations": citations}

print(build_response("answer", citations))
print(build_response("answer", []))


Legacy practice content from practice.ipynb

### Task 4.3: Student fill-in frame

Add your own response contract fields (e.g., confidence, follow_up_questions).


In [ ]:
# Legacy practice content
RESPONSE_FIELDS = """
- confidence:
- follow_up_questions:
- refusal_reason:
"""

print(RESPONSE_FIELDS)


In [ ]:
# Legacy practice content
# TODO: Delete this legacy helper cell if not needed.
# (A newer format_citations implementation is provided above.)
